# Context Manager Applications

- Open File: Built-In
- Open File: Custom Context Manager
- Resource Manager
- DataIterator
- Decimal Precision
- Timer
- Logger
- HTML Tagging
- List Generator

## Open File: Built-In

In [9]:
with open('README.md', 'w') as file:
    print('inside with: file closed?', file.closed)
print('after with: file closed?', file.closed)

inside with: file closed? False
after with: file closed? True


## Open File: Custom Context Manager

In [10]:
class File:
    def __init__(self, name, mode):
        self.name = name
        self.mode = mode
        
    def __enter__(self):
        print('opening file...')
        self.file = open(self.name, self.mode)
        return self.file
    
    def __exit__(self, exc_type, exc_value, exc_traceback):
        print('closing file...')
        self.file.close()
        return False

In [11]:
with File('README.md', 'w') as f:
    f.write('This is some content that we are adding to the file using our own context manager.')

opening file...
closing file...


In [12]:
with File('README.md', 'r') as file_ctx:
    print(next(file_ctx))
    print(file_ctx.name)
    print(file_ctx.mode)

opening file...
This is some content that we are adding to the file using our own context manager.
README.md
r
closing file...


## Resource Manager: Custom Context Manager

### With Exception Handling

In [13]:
class ResourceManager:
    def __init__(self, name):
        self.name = name
        self.resource = None
        
    def __enter__(self):
        print('entering context')
        self.resource = ResourceManager(self.name)
        self.resource.state = 'created'
        return self.resource
    
    def __exit__(self, exc_type, exc_value, exc_traceback):
        print('exiting context')
        self.resource.state = 'destroyed'
        if exc_type:
            print('error occurred')
        return False

In [14]:
with ResourceManager('spam') as res:
    print(f'{res.name} = {res.state}')
print(f'{res.name} = {res.state}')

entering context
spam = created
exiting context
spam = destroyed


###  Without Exception Handling

In [15]:
class ResourceManager:
    def __init__(self, name):
        self.name = name
        self.resource = None
        
    def __enter__(self):
        print('entering context')
        self.resource = ResourceManager(self.name)
        self.resource.state = 'created'
        return self.resource
    
    def __exit__(self, exc_type, exc_value, exc_traceback):
        print('exiting context')
        self.resource.state = 'destroyed'
        if exc_type:
            print('error occurred')
        return True

In [16]:
with ResourceManager('spam') as res:
    print(f'{res.name} = {res.state}')
print(f'{res.name} = {res.state}')

entering context
spam = created
exiting context
spam = destroyed


## DataIterator

In [17]:
class DataIterator:
    def __init__(self, fname):
        self._fname = fname
        self._f = None
    
    def __iter__(self):
        return self
    
    def __next__(self):
        row = next(self._f)
        return row.strip('\n').split(',')
    
    def __enter__(self):
        self._f = open(self._fname)
        return self
    
    def __exit__(self, exc_type, exc_value, exc_tb):
        if not self._f.closed:
            self._f.close()
        return False

In [18]:
with DataIterator('README.md') as data:
    for row in data:
        print(row)

['This is some content that we are adding to the file using our own context manager.']


## Decimal Precision

In [19]:
import decimal

In [20]:
class precision:
    def __init__(self, prec):
        self.prec = prec
        self.current_prec = decimal.getcontext().prec

    def __enter__(self):
        decimal.getcontext().prec = self.prec

    def __exit__(self, exc_type, exc_value, exc_tb):
        decimal.getcontext().prec = self.current_prec
        return False

In [21]:
decimal.getcontext().prec = 14
with precision(3):
    print(decimal.Decimal(1) / decimal.Decimal(3))

print(decimal.Decimal(1) / decimal.Decimal(3))

0.333
0.33333333333333


## Timer

In [22]:
from time import perf_counter, sleep

class Timer:
    def __init__(self):
        self.elapsed = 0

    def __enter__(self):
        self.start = perf_counter()
        return self

    def __exit__(self, exc_type, exc_value, exc_tb):
        self.stop = perf_counter()
        self.elapsed = self.stop - self.start
        return False

In [23]:
with Timer() as timer:
    sleep(1)
print(timer.elapsed)

1.001089998999987


## Logger

In [24]:
import sys

class OutToFile:
    def __init__(self, fname):
        self._fname = fname
        self._current_stdout = sys.stdout
        
    def __enter__(self):
        self._file = open(self._fname, 'w')
        sys.stdout = self._file
        return self
        
    def __exit__(self, exc_type, exc_value, exc_tb):
        sys.stdout = self._current_stdout
        if self._file:
            self._file.close()
        return False

In [25]:
with OutToFile('test.txt') as o:
    print('Line 1')
    print('Line 2')
print('Line 3')
print(o)

Line 3


## HTML Tagging

In [26]:
class Tag:
    def __init__(self, tag):
        self._tag = tag

    def __enter__(self):
        print(f'<{self._tag}>', end = '')

    def __exit__(self, e_t, e_v, e_tr):
        print(f'</{self._tag}>', end='')
        return False

In [27]:
with Tag('p'):
    print('In F1, ', end='')
    with Tag('b'):
        print('Mercedes ', end='')
    print(' is the best team.', end='')

<p>In F1, <b>Mercedes </b> is the best team.</p>

<p>In F1, <b>Mercedes </b> is the best team.</p>

## List Generator

In [28]:
class ListMaker:
    def __init__(self, title, prefix='- ', indent=3):
        self._title = title
        self._prefix = prefix
        self._indent = indent
        self._current_indent = 0
        print(title)
        
    def __enter__(self):
        self._current_indent += self._indent
        return self
    
    def __exit__(self, exc_type, exc_value, exc_tb):
        self._current_indent -= self._indent
        return False
        
    def print(self, arg):
        s = ' ' * self._current_indent + self._prefix + str(arg)
        print(s)

In [29]:
with ListMaker('Items') as lm:
    lm.print('Item 1')
    with lm:
        lm.print('item 1a')
        lm.print('item 1b')
    lm.print('Item 2')
    with lm:
        lm.print('item 2a')
        lm.print('item 2b')

print(lm)

Items
   - Item 1
      - item 1a
      - item 1b
   - Item 2
      - item 2a
      - item 2b


---

## Generators and Context Managers

### Context Manager with Callable Functions as Generators

In [30]:
def my_gen():
    try:
        print('creating context and yeilding some object')
        lst = [1, 2, 3,4, 5, 'y']
        yield lst
    finally:
        print('exiting the context and cleaning up')

In [31]:
gen = my_gen()
lst = next(gen)
print(lst)
try:
    next(gen)
except StopIteration:
    pass

creating context and yeilding some object
[1, 2, 3, 4, 5, 'y']
exiting the context and cleaning up


In [34]:
class GenCtxManager:
    def __init__(self, gen_func, *args, **kwargs):
        self._gen = gen_func(*args, **kwargs)

    def __enter__(self):
        return next(self._gen)

    def __exit__(self, e_t, e_v, e_tr):
        try:
            next(self._gen)
        except StopIteration:
            pass
        return False

In [35]:
with GenCtxManager(my_gen) as lst:
    print(lst)

creating context and yeilding some object
[1, 2, 3, 4, 5, 'y']
exiting the context and cleaning up


### Context Managers for Non-Callable Generators


In [38]:
def open_file(fname, mode='r'):
    print('opening file..')
    f = open(fname, mode)
    try:    
        yield f
    finally:
        print('closing file...')
        f.close()

In [39]:
class GenCtxManager:
    def __init__(self, gen):
        self._gen = gen

    def __enter__(self):
        return next(self._gen)

    def __exit__(self, e_t, e_v, e_tr):
        print('calling the next to perform cleanup in generator')
        try:
            next(self._gen)
        except StopIteration:
            pass
        return False

In [40]:
file_gen = open_file('README.md', 'w')

with GenCtxManager(file_gen) as f:
    f.writelines('This is a new line never seen before')

opening file..
calling the next to perform cleanup in generator
closing file...


## Context Managers, Generators and Decorators

Convert a Generator into Context Manager using Decorator

### Custom

In [41]:
def context_manager_dec(gen_fn):
    def helper(*args, **kwargs):
        gen = gen_fn(*args, **kwargs)
        ctx = GenCtxManager(gen)
        return ctx
    return helper

In [42]:
@context_manager_dec
def open_file(fname, mode='r'):
    print("opening file")
    f = open(fname, mode)
    try:
        yield f

    finally:
        print("closing the file")
        f.close()

In [43]:
with open_file('test.txt') as f:
    print(f.readlines())

opening file
['Line 1\n', 'Line 2\n']
calling the next to perform cleanup in generator
closing the file


### Built-In

In [45]:
from contextlib import contextmanager
from time import perf_counter, sleep

In [46]:
@contextmanager
def timer():
    stats = dict()
    start = perf_counter()
    stats['start'] = start
    try:
        yield stats
    finally:
        end = perf_counter()
        stats['end'] = end
        stats['elapsed'] = end - start

In [47]:
with timer() as stats:
    sleep(1)

In [48]:
stats

{'elapsed': 1.0006277180000325, 'end': 2943.199498652, 'start': 2942.198870934}